In [154]:
import networkx as nx
import random

# for write dictionaries for txt file
import json
import copy

In [2]:
G = nx.read_edgelist("dataset_ubicomp2013_tips_final_subcompontents_removed.txt")

Simple sampling, we can skip this part

In [15]:
k = 1000
sampled_nodes = random.sample(G.nodes, k)
sampled_graph = G.subgraph(sampled_nodes)

C:\Users\MUSIC\AppData\Local\Temp/ipykernel_13112/75908891.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sampled_nodes = random.sample(G.nodes, k)


Ther was a problem with a "freezed" graph, that can means kind a protection, so code unfreeze it to be able to remove small clusters

In [4]:
unfrozen_graph = nx.Graph(sampled_graph)
nx.is_frozen(unfrozen_graph)

NameError: name 'sampled_graph' is not defined

Below needs after the sampling, since small clusters have in the graphs

In [17]:
max([len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)])

167

In [18]:
compfix = [len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)]
print("components in the dataset: ", len(compfix))
print([len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)])
biggest_comp = max([len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)])

components in the dataset:  685
[167, 12, 7, 7, 7, 6, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Remove clusters which are smaller than the biggest one

In [19]:
# remove the sub clusters

for component in list(nx.connected_components(unfrozen_graph)):
    if len(component)<biggest_comp:
        for node in component:
            unfrozen_graph.remove_node(node)

Double check

In [20]:
compfix = [len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)]
print("components in the dataset: ", len(compfix))
print([len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)])

components in the dataset:  1
[167]


Preferential Attachment method for the new connections

In [3]:
linkpred_pa = list(nx.preferential_attachment(unfrozen_graph))

linkpred_pa.sort(key = lambda tup: tup[2], reverse = True)

NameError: name 'unfrozen_graph' is not defined

# For the original dataset

In [ ]:
linkpred_pa = list(nx.preferential_attachment(unfrozen_graph))

linkpred_pa.sort(key = lambda tup: tup[2], reverse = True)

Here we just neglect the lines, where user connect to user or venue to venue.

In [22]:
recommends_list_pa = []
for i in range(len(linkpred_pa)):
    if (linkpred_pa[i][0][0] == "u" and linkpred_pa[i][1][0] == "v") or (linkpred_pa[i][0][0] == "v" and linkpred_pa[i][1][0] == "u"):
        recommends_list_pa.append(linkpred_pa[i])
    else:
        None
recommends_list_pa[:20]

[('u72', 'v16020', 80),
 ('v15139', 'u72', 64),
 ('u72', 'v7854', 64),
 ('u27404', 'v6293', 54),
 ('v4843', 'u72', 48)]

# With the big dataset

In [75]:
G = nx.read_edgelist("dataset_ubicomp2013_tips_final_subcompontents_removed.txt")

In [126]:
# prepare for a dictionary to check recomandations / user
u_s = set()
user_recommandations_dictionary = {}
for e in G.edges():
    if e[0][0] == "u":
        u_s.add(e[0])
    if e[1][0] == "u":
        u_s.add(e[1])
#convert into a list
l_u = list(u_s)
# loop thorugh the list to add the keys for the dictionary
for i in l_u:
    user_recommandations_dictionary[i] = []

In [127]:
linkpred_pa = list(nx.preferential_attachment(G))

linkpred_pa.sort(key = lambda tup: tup[2], reverse = True)

In [128]:
# Fasten the code with early for the acces by index
recommends_list_pa = []
#for i in range(len(linkpred_pa)):
for i in range(len(linkpred_pa)):
    #print(linkpred_pa[i])
    v = linkpred_pa[i]
    #print(v[0][0], "  ", v[1][0], v[2])
    if (v[0][0] == "u" and v[1][0] == "v"):
        r = [v[1], v[2]]
        #print(r)
        recommends_list_pa.append(linkpred_pa[i])
        user_recommandations_dictionary[v[0]].append(r)
    if (v[0][0] == "v" and v[1][0] == "u"):
        r = [v[0], v[2]]
        recommends_list_pa.append(linkpred_pa[i])
        user_recommandations_dictionary[v[1]].append(r)
    else:
        None
#recommends_list_pa[:100]

Since the recomandation file is about 104MB, which is huge, we should do some shortcut - lets say, we will focus on the first 20 results

In [149]:
with open('user_recommandations_dictionary_pa.txt', 'w') as convert_file:
     convert_file.write(json.dumps(user_recommandations_dictionary))

In [159]:
user_recommandations_dictionary_20_rec = user_recommandations_dictionary.copy()

In [163]:
user_recommandations_dictionary_20_rec

{'u13298': [['v3184', 48],
  ['v39242', 44],
  ['v11100', 44],
  ['v18222', 42],
  ['v564', 40],
  ['v12833', 40],
  ['v17741', 40],
  ['v45403', 40],
  ['v51231', 38],
  ['v1086', 38],
  ['v17698', 38],
  ['v985', 38],
  ['v3218', 36],
  ['v654', 36],
  ['v6293', 36],
  ['v5744', 34],
  ['v5918', 34],
  ['v476', 34],
  ['v7723', 34],
  ['v11954', 34]],
 'u135005': [['v3184', 96],
  ['v39242', 88],
  ['v11100', 88],
  ['v18222', 84],
  ['v564', 80],
  ['v12833', 80],
  ['v17741', 80],
  ['v45403', 80],
  ['v17698', 76],
  ['v985', 76],
  ['v1086', 76],
  ['v51231', 76],
  ['v3218', 72],
  ['v654', 72],
  ['v6293', 72],
  ['v5918', 68],
  ['v11954', 68],
  ['v5744', 68],
  ['v7723', 68],
  ['v476', 68]],
 'u136058': [['v3184', 72],
  ['v39242', 66],
  ['v11100', 66],
  ['v18222', 63],
  ['v564', 60],
  ['v12833', 60],
  ['v17741', 60],
  ['v45403', 60],
  ['v17698', 57],
  ['v985', 57],
  ['v51231', 57],
  ['v1086', 57],
  ['v3218', 54],
  ['v654', 54],
  ['v6293', 54],
  ['v5918', 51],

In [161]:
for key in user_recommandations_dictionary_20_rec.keys():
    user_recommandations_dictionary_20_rec[key] = user_recommandations_dictionary_20_rec[key][0:20]

In [162]:
with open('user_recommandations_dictionary_pa_20_rec.txt', 'w') as convert_file:
     convert_file.write(json.dumps(user_recommandations_dictionary_20_rec))

# Second run for the other link predictors

## Jacard - it takes a five mintes, and gives back recommandations with 0

In [164]:
linkpred_ja = list(nx.jaccard_coefficient(G))

In [165]:
linkpred_ja.sort(key = lambda tup: tup[2], reverse = True)

In [166]:
# use the dictionary, empty it
for i in l_u:
    user_recommandations_dictionary[i] = []

### Below code is running for a while 

In [167]:
# Fasten the code with early for the acces by index
recommends_list_ja = []
for i in range(len(linkpred_ja)):
    #print(linkpred_pa[i])
    v = linkpred_ja[i]
    #print(v[0][0], "  ", v[1][0], v[2])
    if (v[0][0] == "u" and v[1][0] == "v"):
        r = [v[1], v[2]]
        #print(r)
        recommends_list_ja.append(linkpred_ja[i])
        user_recommandations_dictionary[v[0]].append(r)
    if (v[0][0] == "v" and v[1][0] == "u"):
        r = [v[0], v[2]]
        recommends_list_ja.append(linkpred_ja[i])
        user_recommandations_dictionary[v[1]].append(r)
    else:
        None

In [168]:
user_recommandations_dictionary_20_rec = user_recommandations_dictionary.copy()

In [171]:
# I used only 1 results here to check whats going on
for key in user_recommandations_dictionary_20_rec.keys():
    user_recommandations_dictionary_20_rec[key] = user_recommandations_dictionary_20_rec[key][0:1]

In [172]:
user_recommandations_dictionary_20_rec

{'u13298': [['v504', 0.0]],
 'u135005': [['v504', 0.0]],
 'u136058': [['v504', 0.0]],
 'u96969': [['v504', 0.0]],
 'u22658': [['v504', 0.0]],
 'u42406': [['v504', 0.0]],
 'u25472': [['v504', 0.0]],
 'u2022': [['v504', 0.0]],
 'u115700': [['v504', 0.0]],
 'u227224': [['v504', 0.0]],
 'u2163': [['v504', 0.0]],
 'u27559': [['v504', 0.0]],
 'u10798': [['v504', 0.0]],
 'u5748': [['v504', 0.0]],
 'u4670': [['v504', 0.0]],
 'u4730': [['v504', 0.0]],
 'u21920': [['v504', 0.0]],
 'u4766': [['v504', 0.0]],
 'u19729': [['v504', 0.0]],
 'u82682': [['v504', 0.0]],
 'u149548': [['v504', 0.0]],
 'u23999': [['v504', 0.0]],
 'u1317': [['v504', 0.0]],
 'u68202': [['v504', 0.0]],
 'u44309': [['v504', 0.0]],
 'u80795': [['v504', 0.0]],
 'u28946': [['v504', 0.0]],
 'u45533': [['v504', 0.0]],
 'u130957': [['v504', 0.0]],
 'u8810': [['v504', 0.0]],
 'u10448': [['v504', 0.0]],
 'u41031': [['v504', 0.0]],
 'u217242': [['v504', 0.0]],
 'u248947': [['v504', 0.0]],
 'u38431': [['v504', 0.0]],
 'u18863': [['v504',

## AA - who knows? - its running also for a while

In [173]:
linkpred_aa = list(nx.adamic_adar_index(G))

In [174]:
linkpred_aa.sort(key = lambda tup: tup[2], reverse = True)

In [175]:
# use the dictionary, empty it
for i in l_u:
    user_recommandations_dictionary[i] = []
    
# Fasten the code with early for the acces by index
recommends_list_aa = []
for i in range(len(linkpred_aa)):
    #print(linkpred_pa[i])
    v = linkpred_aa[i]
    #print(v[0][0], "  ", v[1][0], v[2])
    if (v[0][0] == "u" and v[1][0] == "v"):
        r = [v[1], v[2]]
        #print(r)
        recommends_list_ja.append(linkpred_aa[i])
        user_recommandations_dictionary[v[0]].append(r)
    if (v[0][0] == "v" and v[1][0] == "u"):
        r = [v[0], v[2]]
        recommends_list_ja.append(linkpred_aa[i])
        user_recommandations_dictionary[v[1]].append(r)
    else:
        None

user_recommandations_dictionary_20_rec = user_recommandations_dictionary.copy()

In [176]:
for key in user_recommandations_dictionary_20_rec.keys():
    user_recommandations_dictionary_20_rec[key] = user_recommandations_dictionary_20_rec[key][0:10]

In [177]:
user_recommandations_dictionary_20_rec

{'u13298': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u135005': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u136058': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u96969': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u22658': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u42406': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0

# RA

In [178]:
# Run all of the link prediction methods implemented in networkx

linkpred_ra = list(nx.resource_allocation_index(G))


linkpred_ra.sort(key = lambda tup: tup[2], reverse = True)


In [180]:
for i in l_u:
    user_recommandations_dictionary[i] = []
    
# Fasten the code with early for the acces by index
recommends_list_ra = []
for i in range(len(linkpred_ra)):
    #print(linkpred_pa[i])
    v = linkpred_ra[i]
    #print(v[0][0], "  ", v[1][0], v[2])
    if (v[0][0] == "u" and v[1][0] == "v"):
        r = [v[1], v[2]]
        #print(r)
        recommends_list_ra.append(linkpred_ra[i])
        user_recommandations_dictionary[v[0]].append(r)
    if (v[0][0] == "v" and v[1][0] == "u"):
        r = [v[0], v[2]]
        recommends_list_ra.append(linkpred_ra[i])
        user_recommandations_dictionary[v[1]].append(r)
    else:
        None

user_recommandations_dictionary_20_rec = user_recommandations_dictionary.copy()

for key in user_recommandations_dictionary_20_rec.keys():
    user_recommandations_dictionary_20_rec[key] = user_recommandations_dictionary_20_rec[key][0:10]

In [181]:
user_recommandations_dictionary_20_rec

{'u13298': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u135005': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u136058': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u96969': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u22658': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0],
  ['v9727', 0],
  ['v42059', 0],
  ['v717', 0]],
 'u42406': [['v504', 0],
  ['v3449', 0],
  ['v33031', 0],
  ['v12281', 0],
  ['v10224', 0],
  ['v86184', 0],
  ['v71417', 0